# DATOS - ARGENTINA

**En este notebook se llegará al dataframe final para luego seguir con análsis exploratorio y feature engeneering.**

In [66]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore') # Para evitar los molestos avisos.
%matplotlib inline

Lectura de datos demográficos de Argentina

In [67]:
path = 'C:/Users/dlongas/Documents/Personal/MIMYANALYTICS/TESIS/files/AR/AR_demo_202101_grouped/final_csv/'
columns_to_keep = ['type','user_id','0','1']
#abro un csv y luego le pego los demas
df_demo = pd.read_csv(path + 'AR_demo_202101_0.csv', usecols=columns_to_keep)
mylist = [f for f in glob.glob(path + "AR_demo_*.csv")]
for file in mylist:
    df = pd.read_csv(file, usecols=columns_to_keep)
    df_demo = df_demo.append(df)
df_demo = df_demo.drop_duplicates()
df_demo.head(2)

,type,user_id,0,1
0,web,00028b8f-e4ff-432b-aad3-bf9b6f133836,5,2.0
1,web,0105a039-237e-4cad-b765-c4850423438e,2,6.0


Lectura de datos de features de Argentina

In [69]:
path = 'C:/Users/dlongas/Documents/Personal/MIMYANALYTICS/TESIS/files/AR/feature_index/final_csv/'
columns_to_keep = ['feat_idx','feature']
#abro un csv
df_feature = pd.read_csv(path + 'AR_feature_index_0.csv', usecols=columns_to_keep) 
print(df_feature.shape[0], ' features en total para Argentina')
df_feature.head(2)

32552  features en total para Argentina


,feat_idx,feature
0,0,bf@Brand_browser
1,1,bf@Chrome


Lectura de identificadores de features y de dispositivos para Argentina

In [71]:
path = 'C:/Users/dlongas/Documents/Personal/MIMYANALYTICS/TESIS/files/AR/indexed/final_csv/'
columns_to_keep = ['device_idx','feat_idx']
#abro un csv y luego le pego los demas
df_dev_fea = pd.read_csv(path + 'AR_parquet_indexed_0.csv', usecols=columns_to_keep)
mylist = [f for f in glob.glob(path + "AR_parquet_indexed_*.csv")]
for file in mylist:
    df = pd.read_csv(file, usecols=columns_to_keep)
    df_dev_fea = df_dev_fea.append(df)
df_dev_fea = df_dev_fea.drop_duplicates()
print("Hay ",df_dev_fea.shape[0], " combinaciones distintas de dispositivos y features para Argentina" )
df_dev_fea.head(2)

Hay  2516240  combinaciones distintas de dispositivos y features para Argentina


,device_idx,feat_idx
0,40990,18802
1,7711,23092


Descarga de dispositivos y su identificador para Argentina

In [73]:
path = 'C:/Users/dlongas/Documents/Personal/MIMYANALYTICS/TESIS/files/AR/device_index/final_csv/'
columns_to_keep = ['device_idx','device_id']
#abro un csv y luego le pego los demas
df_dev_id = pd.read_csv(path + 'AR_device_index_0.csv', usecols=columns_to_keep)
print("Hay ", df_dev_id.shape[0], " dispositivos (usuarios) en Argentina")
df_dev_id.head(2)

Hay  101990  dispositivos (usuarios) en Argentina


,device_idx,device_id
0,0,00006070-10bc-44e8-bf08-e4912614de23
1,1,0000ea8d-feb4-4166-bee6-a51991c477e6


Unifico feature id - device id - feature name para Argentina

In [75]:
df = pd.merge(df_dev_fea, df_feature, how='left', on='feat_idx')
df = pd.merge(df, df_dev_id, how='left', on='device_idx')
print("Reafirmamos las ", df.shape[0], 'combinaciones distintinas de dispositivos y features para Argentina')
df.head(2)

Reafirmamos las  2516240 combinaciones distintinas de dispositivos y features para Argentina


,device_idx,feat_idx,feature,device_id
0,40990,18802,url@tuenti.com.ar/recargar-combo-tuenti,5c6f34ad-d15d-4b19-b278-6f9a534a7602
1,7711,23092,url@zonajobs.com.ar/empleos/jefe-relaciones-la...,148fc89c-1781-4d3f-a852-8b8fa02e6190


Unifico feature id - device id - feature name para Chile

In [76]:
df_cl = pd.merge(df_dev_fea_cl, df_feature_cl, how='left', on='feat_idx')
df_cl = pd.merge(df_cl, df_dev_id_cl, how='left', on='device_idx')
print("Reafirmamos las ", df_cl.shape[0], 'combinaciones distintinas de dispositivos y features para Chile')
df_cl.head(2)

Reafirmamos las  13990011 combinaciones distintinas de dispositivos y features para Chile


,device_idx,feat_idx,feature,device_id
0,3958,28829,url@pagos.movistar.cl/comprobante,011ffb5f-34ee-47b3-bece-0f82752f51e1
1,4982,3306,im@1,01694d00-0586-4086-bbd9-918880df5870


Dispositivos y Features para Argentina

In [77]:
df_analisis_exp = df[['device_idx', 'feature']]
print('Tomamos los', df_analisis_exp.shape[0], 'dispositivos-features (Arg)')
df_analisis_exp.head(2)

Tomamos los 2516240 dispositivos-features (Arg)


,device_idx,feature
0,40990,url@tuenti.com.ar/recargar-combo-tuenti
1,7711,url@zonajobs.com.ar/empleos/jefe-relaciones-la...


Features en Argentina

In [78]:
expand_feature = df_analisis_exp['feature'].str.split('@', expand=True)
#expand_feature[expand_feature[2]=='gmail.com-ortega-1909843.html']
expand_feature = expand_feature.drop([2], axis=1)
expand_feature = expand_feature.rename(columns={0:'feature_type',
                                                1:'feature_detail'})
print("Separo el campo feature para obtener el tipo de feature por un lado y el detalle por otro")
print(expand_feature.shape[0], 'dispositivos - features en Arg')
print("Tipos de features (Arg)")
expand_feature['feature_type'].unique()

expand_feature.head(2)

Separo el campo feature para obtener el tipo de feature por un lado y el detalle por otro
2516240 dispositivos - features en Arg
Tipos de features (Arg)


,feature_type,feature_detail
0,url,tuenti.com.ar/recargar-combo-tuenti
1,url,zonajobs.com.ar/empleos/jefe-relaciones-labora...


Dataframe Argentina

In [79]:
df_analisis_exp = pd.concat([df_analisis_exp, expand_feature], axis=1)
df_analisis_exp = df_analisis_exp[['device_idx', 'feature_type', 'feature_detail']]
print('shape del df:', df_analisis_exp.shape[0])
df_analisis_exp['country'] = 'AR'
df_analisis_exp.head(3)

shape del df: 2516240


,device_idx,feature_type,feature_detail,country
0,40990,url,tuenti.com.ar/recargar-combo-tuenti,AR
1,7711,url,zonajobs.com.ar/empleos/jefe-relaciones-labora...,AR
2,79625,url,bumeran.com.ar/empleos/administrativo-caba-rec...,AR


Reordenamiento de Labels en dataset Argentina

In [83]:
#Veo como están las clases (género y edad)
#Modifico mi df
df_demo_0 = df_demo[['user_id','0']]
df_demo_0 = df_demo_0.rename(columns={'0':'label'})
df_demo_0 = df_demo_0.drop_duplicates()
df_demo_1 = df_demo[['user_id','1']]
df_demo_1 = df_demo_1.rename(columns={'1':'label'})
df_demo_1 = df_demo_1.drop_duplicates()
print(df_demo_0.shape)
print(df_demo_0.head())
print(df_demo_1.shape)
print(df_demo_1.head())
df_demo_exp_ar = pd.concat([df_demo_0,df_demo_1], axis=0)
df_demo_exp_ar = df_demo_exp_ar[df_demo_exp_ar['label']!=1]
df_demo_exp_ar['country'] = 'AR'
df_demo_exp_ar = pd.merge(df_demo_exp_ar, df_dev_id, how='left', left_on='user_id', right_on='device_id')
df_demo_exp_ar = df_demo_exp_ar[~df_demo_exp_ar['device_idx'].isnull()]
df_demo_exp_ar = df_demo_exp_ar.drop_duplicates()
df_demo_exp_ar = df_demo_exp_ar[['device_idx', 'label', 'country']]
print(df_demo_exp_ar.shape[0], "dispositivos con lebel de genero y/o de edad para ARGENTINA") #tiene que haber 101990
df_demo_exp_ar.head(2)

(103908, 2)
                                user_id  label
0  00028b8f-e4ff-432b-aad3-bf9b6f133836      5
1  0105a039-237e-4cad-b765-c4850423438e      2
2  01c8e44a-d32e-4143-be36-a592dcdf11e6      6
3  026cdff1-082f-4e97-8fe4-8cdef8ee2370      2
4  028a5c57-2a93-41f1-8086-87b6b9c92cc5      5
(103908, 2)
                                user_id  label
0  00028b8f-e4ff-432b-aad3-bf9b6f133836    2.0
1  0105a039-237e-4cad-b765-c4850423438e    6.0
2  01c8e44a-d32e-4143-be36-a592dcdf11e6    3.0
3  026cdff1-082f-4e97-8fe4-8cdef8ee2370    5.0
4  028a5c57-2a93-41f1-8086-87b6b9c92cc5    3.0
199490 dispositivos con lebel de genero y/o de edad para ARGENTINA


,device_idx,label,country
0,4.0,5.0,AR
1,392.0,2.0,AR


Se guarda el dataset

In [86]:
df_demo_exp_ar.to_csv('df_demo_exp_ar.csv')

*El dataset final con datos de Argentina y Chile se encuentrá generado en la última sección del notebook "DATOS - CHILE"*